# imports

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# cargar el dataset

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns= iris.feature_names)
df['flower'] = iris.target

df.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
91,6.1,3.0,4.6,1.4,1
76,6.8,2.8,4.8,1.4,1
134,6.1,2.6,5.6,1.4,2
80,5.5,2.4,3.8,1.1,1
16,5.4,3.9,1.3,0.4,0


# split y hold out

In [3]:
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = df['flower']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.values)
X_test = scaler.transform(X_test.values)

# SVM

class sklearn.svm.SVR(*, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)[source]¶

- **C** float, default=1.0

    Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive.     The penalty is a squared l2 penalty.


- **kernel** = {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}

    Specifies the kernel type to be used in the algorithm. If none is given, ‘rbf’ will be used. If a callable is given it     is used to pre-compute the kernel matrix from data matrices; that matrix should be an array of shape 
    
    
- **gamma** = {'scale', 'auto'} or float, default='scale'

    Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.

    if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,

     if ‘auto’, uses 1 / n_features

     if float, must be non-negative.
     
     
- **decision_function_shape** = {'ovo', 'ovr'}, default='ovr'

    Whether to return a one-vs-rest (‘ovr’) decision function of shape (n_samples, n_classes) as all other classifiers, or     the original one-vs-one (‘ovo’) decision function of libsvm which has shape (n_samples, n_classes * (n_classes - 1) /       2). However, note that internally, one-vs-one (‘ovo’) is always used as a multi-class strategy to train models; an ovr     matrix is only constructed from the ovo matrix. The parameter is ignored for binary classification.

In [4]:
hiperparams = {'C': [0.1 ,1, 10, 20, 50],
               'kernel': ['rbf', 'linear']}

grid_search_SVC = GridSearchCV(estimator= SVC(gamma= 'auto'),                      # el modelo
                          param_grid= hiperparams,                                 # red de hiperparametros
                          scoring='accuracy',                                      # metrica a optimizar
                          cv= KFold(n_splits= 5, shuffle= True, random_state=42),  # cross validation interna
                          n_jobs= -1,                                              # usa todos los nucleos de la cpu
                         ) 
grid_search_SVC.fit(X_train, y_train)

print(grid_search_SVC.best_estimator_)
print(grid_search_SVC.best_params_)

SVC(C=1, gamma='auto', kernel='linear')
{'C': 1, 'kernel': 'linear'}


In [14]:
metricas = {
  'ACC':    lambda y_true, y_pred: metrics.accuracy_score(y_true, y_pred),
  'PREC':   lambda y_true, y_pred:metrics.precision_score(y_true, y_pred, average= 'macro'),
  'RECALL': lambda y_true, y_pred:
            metrics.recall_score(y_true, y_pred, average= 'macro'),
  'F1':     lambda y_true, y_pred:
            metrics.f1_score(y_true, y_pred, average= 'macro'),
#  "AUC":    lambda y_true, y_pred: metrics.roc_auc_score(y_true, y_pred)
}
# hiperparams = {'C': [0.1 ,1, 10, 20, 50],
#                'kernel': ['rbf', 'linear']}


# # para mas de una metrica
# grid_search_SVC = GridSearchCV(estimator= SVC(gamma= 'auto'),                      # el modelo
#                           param_grid= hiperparams,                                 # red de hiperparametros
#                           scoring=metricas,                                        # metrica a optimizar
#                           cv= KFold(n_splits= 5, shuffle= True, random_state=42),  # cross validation interna
#                           n_jobs= -1,                                              # usa todos los nucleos de la cpu
#                           refit= 'F1'
#                          ) 
# grid_search_SVC.fit(X_train, y_train)

# print(grid_search_SVC.best_estimator_)
# print(grid_search_SVC.best_params_)

In [18]:
from sklearn import metrics
best_SVC_model = grid_search_SVC.best_estimator_

best_SVC_model.fit(X_train, y_train)

y_pred = best_SVC_model.predict(X_test)

for nombre, metrica in metricas.items():
    print(f'para la metrica {nombre} \t  se obtienen un valor de \t  ---> \t {metrica(y_pred, y_test)}')

para la metrica ACC 	  se obtienen un valor de 	  ---> 	 0.9777777777777777
para la metrica PREC 	  se obtienen un valor de 	  ---> 	 0.9743589743589745
para la metrica RECALL 	  se obtienen un valor de 	  ---> 	 0.9761904761904763
para la metrica F1 	  se obtienen un valor de 	  ---> 	 0.974320987654321
